# step1 导入依赖

In [3]:
from pickletools import optimize

from torch.utils.data.datapipes.map.callable import default_fn
from transformers import AutoTokenizer,AutoModelForSequenceClassification

# step2 数据清洗

In [6]:
import pandas as pd

# 指定数据集
data = pd.read_csv("./data/ChnSentiCorp_htl_all.csv")

# 删除空数据
data = data.dropna()

# step3 创建Dataset

In [7]:
from torch.utils.data import Dataset

# 创建一个用于读取与清晰数据的类
class MyDataset(Dataset):
    
    def __init__(self) -> None:
        super().__init__()
        self.data = pd.read_csv("./data/ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()
        
    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    
    def __len__(self):
        return len(self.data)

In [8]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


# step5 创建Dataloader

In [9]:
import torch
tokenizer = AutoTokenizer.from_pretrained('./model/rbt3')

def collate_func(batch):
    texts,labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

In [10]:
from torch.utils.data import random_split

train_set, valid_set = random_split(dataset, lengths=[0.9, 0.1])
len(train_set), len(valid_set)

(6989, 776)

In [11]:
for i in range(10):
    print(train_set[i], valid_set[i])

('免费接机用的轿车。宽带上网只是一个摆设，根本不能上网，叫了几次前台也没有解决，最后只能用电话线拨号。我住的客房面对公路，隔音效果不好，早上卡车经过很吵。晚上有小姐骚扰电话。对面有韩国旅行团入住，小姐进出频繁。听朋友说那里的小姐比较宰客，谈好的价钱只是台费，最后还会另收天价小费，不给的话，会有鸡头进来跟你谈，挺恐怖的，最好不要尝试。', 0) ('又涨价了！价格极端不稳定，不在携程订了，直接去前台。', 1)
('1、总体感觉有些旧，地毯上污迹比较多。2、服务等还可以。3、房间总带有一些烟味，不喜欢。', 1) ('很实惠的酒店~~这次住的是经济房，没有独立的卫生间，但是房间很干净，没有提供一次性的洗濑用品，需要自带，很适合背包一族。位置也很好，离火车站很近，在二环内，在北京算是很便宜的酒店了~~~服务一般，不过主要是旅游，所以只要住的干净也没关系~~~总体还是不错的！', 1)
('服务很一般房间还可以条件比较一般价格没竞争力', 0) ('大堂竟然没有电源插孔可以充电，服务还算可以，但是服务的意识还是有点欠缺。。。', 1)
('没什么啦,如果你要感受大上海繁华时尚的一面,这里无疑是最佳的了。信价比还可以吧，毕竟这么黄金的地段而且交通超级方便，紧邻一号地铁的出口，几乎不费什么力就可以到达宾馆。我住过好几次了，不过打折下来568元的价位还是有点“吃血”的，所以也只敢在荷包比较鼓的时候，呵呵，比如发年终奖的这会儿，奢侈一回。宾馆内部设施干净、简洁，是我喜欢的那种，不过公共部位的空间狭小了一些，有“螺蛳壳里做道场”的感觉。毕竟，寸土寸金的地儿，可以理解。推荐如果你是SHOPPING狂人的话，这里很合适，左右前后都是店，还整天搞活动，你可以逛累了，回宾馆歇会儿再杀出去，哈哈！', 1) ('已经住过多次了，还是一个字：好！服务好，环境好！服务人性化，房间虽然小些，但对我足够了。一点儿遗憾是没有免费的矿泉水。早餐不错，很丰盛尤其是有一些小笼，像南方的早茶了。对于3星级有此早餐很好了。携程的价格还是稍差一些，我以其他公司协议价住过要更便宜些。总的讲：下次还住，还推荐其它同事住。', 1)
('房间环境不错，前台礼貌服务也很到位，好评！', 1) ('很不错的酒店，环境优美，设施也很新。可以看到东湖，周围交通也比较方便。要注意的是其实离武大本部还是挺远的。', 1)
('第

In [12]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=train_set, batch_size=32, shuffle=True, collate_fn=collate_func)
valid_loader = DataLoader(dataset=valid_set, batch_size=64, shuffle=False, collate_fn=collate_func)

In [13]:
next(enumerate(train_loader))[1]

{'input_ids': tensor([[ 101, 1962, 6496,  ...,    0,    0,    0],
        [ 101, 6820, 1377,  ...,    0,    0,    0],
        [ 101, 2697, 6230,  ..., 3717, 8024,  102],
        ...,
        [ 101, 2595,  817,  ...,    0,    0,    0],
        [ 101,  671, 1368,  ...,    0,    0,    0],
        [ 101, 6983, 2421,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0,
        1, 1, 0, 1, 1, 1, 0, 0])}

# 创建模型以及优化器

In [16]:
from torch.optim import Adam
model = AutoModelForSequenceClassification.from_pretrained('./model/rbt3')
if torch.cuda.is_available():
    model = model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./model/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
optimizer = Adam(model.parameters(), lr=2e-5)

# 训练

In [19]:
# 验证率准确率
def evaluate():
    model.eval()
    acc_num = 0
    with torch.no_grad():
        for bath in valid_loader:
            if torch.cuda.is_available():
                batch = {k:v.cuda() for k,v in bath.items()}
            output = model(**batch)
            pred = output.logits.argmax(dim=1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(valid_set)

# 训练
def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in train_loader:
            if torch.cuda.is_available():
                batch = {k:v.cuda() for k,v in batch.items()}
                
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep:{ep}, global_step:{global_step}, loss:{output.loss.item()}")
            global_step += 1
                
        acc = evaluate()
        print(f"ep:{ep}, acc:{acc}")
        

In [20]:
train()

ep:0, global_step:0, loss:0.7595826387405396
ep:0, global_step:100, loss:0.32574647665023804
ep:0, global_step:200, loss:0.24471163749694824
ep:0, acc:0.876288652420044
ep:1, global_step:300, loss:0.22833997011184692
ep:1, global_step:400, loss:0.15852518379688263
ep:1, acc:0.8917525410652161
ep:2, global_step:500, loss:0.11434641480445862
ep:2, global_step:600, loss:0.06591776758432388
ep:2, acc:0.894329845905304


# step 模型预测

In [23]:
sen = "我觉得这家酒店很好，点赞"
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k:v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入:{sen}\n模型预测结果:{pred}")

输入:我觉得这家酒店很好，点赞
模型预测结果:tensor([1], device='cuda:0')
